In [1]:
!pip install pytube3 moviepy ffmpeg-python
!pip install yt-dlp
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
import os
from pytube import YouTube
from google.colab import drive
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from scipy.io import wavfile
import numpy as np
import subprocess
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [23]:
from google.colab import drive
drive.mount('/content/drive')

# Load CSV data
#Define the file path for the CSV file with the required data
file_path = '/content/drive/My Drive/VideoData/S1/VideoData.csv'
#define the output directory , if not a default directory named videoProcessing will be created
output_dir = '/content/drive/My Drive/VideoProcessing0'
video_data_df = pd.read_csv(file_path)
video_data_df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Video link,Source,Language,Dialect,Timestamp,Emotion,Age,Gender,Scrapped data link,SNR Ratio,Sample Rate,Remark
0,https://www.youtube.com/live/YAcRXrrEFZQ?si=go...,Youtube,Hindi,NaN,0:00 - 1:48,Excitement,Middle Aged,Male,NaN,NaN,NaN,NaN
1,https://youtu.be/fPlqjhYyJyg?si=MuMv-gZs9uIByfxq,Youtube,Hindi,NaN,0:00 - 1:56,Excitement,Senior Citizens,Male,NaN,NaN,NaN,NaN
2,https://youtu.be/yJktcDKcNiE?si=qY0VnSQkiPAucKMi,Youtube,Hindi,NaN,0:00 - 0:12,Happiness,Middle Aged,Female,NaN,NaN,NaN,NaN
3,https://youtu.be/Vwxft_tqrcg?si=vObRYn9uMAvehQdl,Youtube,Hindi,NaN,0:00 - 0:58,Excitement,Young Adults,Female,NaN,NaN,NaN,NaN
4,https://youtu.be/kRSEYm_dfwE?si=cTO8W3_DFj5WRdvJ,Youtube,Hindi,NaN,0:00 - 1:00,Excitement,Young Adults,Female,NaN,NaN,NaN,NaN


In [24]:
import os
import pandas as pd
import numpy as np
import subprocess
from yt_dlp import YoutubeDL
from scipy.io import wavfile

#FINAL SCRIIPT, BEST AND FAST TOO

video_data_df = pd.read_csv(file_path)

# Function to calculate RMS (Root Mean Square)
def calculate_rms(signal):
    return np.sqrt(np.mean(signal**2))

# Function to extract SNR for a specific audio segment
def extract_snr(audio_file_path, start_time, end_time):
    rate, data = wavfile.read(audio_file_path)
    start_sample = int(start_time * rate)
    end_sample = int(end_time * rate)
    segment = data[start_sample:end_sample]

    # Signal power is the RMS of the segment
    signal_power = calculate_rms(segment)

    # Noise power is the RMS of the difference between the segment and its mean
    noise_power = calculate_rms(segment - np.mean(segment))

    if noise_power == 0:
        snr = float('inf')
    else:
        snr = 10 * np.log10(signal_power / noise_power)

    return snr, rate

# Function to download and trim video segments
def download_and_trim_video(url, start_time, end_time, output_folder):
    output_video_path = os.path.join(output_folder, 'video.mp4')
    ydl_opts = {
        'format': 'best',
        'outtmpl': '-'
    }

    start_time_str = f"{start_time // 60}:{start_time % 60:02d}"
    end_time_str = f"{end_time // 60}:{end_time % 60:02d}"

    with YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)
        video_url = info_dict['url']

        trim_command = [
            'ffmpeg', '-y', '-ss', start_time_str, '-to', end_time_str,
            '-i', video_url, '-c:v', 'copy', '-c:a', 'copy', output_video_path
        ]

        result = subprocess.run(trim_command, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"Error trimming video: {result.stderr}")
            raise RuntimeError("Failed to trim video")

    return output_video_path

# Function to convert timestamp from MM:SS format to seconds
def timestamp_to_seconds(timestamp):
    try:
        minutes, seconds = map(int, timestamp.split(':'))
        return minutes * 60 + seconds
    except ValueError as e:
        print(f"Error converting timestamp '{timestamp}': {e}")
        raise

def main():
    output_base_path = output_dir  # Adjust the directory as needed
    if not os.path.exists(output_base_path):
        os.makedirs(output_base_path)

    results = []

    for index, row in video_data_df.iterrows():
        youtube_url = row['Video link']
        timestamps = row['Timestamp'].split(' - ')               #Validate if the input csv is split with spaces or without accirdingly adjust this line

        try:
            start_time = timestamp_to_seconds(timestamps[0].strip())
            end_time = timestamp_to_seconds(timestamps[1].strip())
        except ValueError:
            print(f"Skipping video at row {index} due to timestamp conversion error.")
            continue

        output_path = os.path.join(output_base_path, f'video_{index+1}')
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        try:
            # Step 1: Download and trim the video segment
            trimmed_video_path = download_and_trim_video(youtube_url, start_time, end_time, output_path)

            # Step 2: Extract SNR and sampling rate from the trimmed audio
            audio_path = trimmed_video_path.replace('.mp4', '.wav')
            convert_command = f'ffmpeg -y -i "{trimmed_video_path}" -vn -acodec pcm_s16le -ar 44100 "{audio_path}"'
            subprocess.run(convert_command, shell=True, capture_output=True, text=True)

            snr, rate = extract_snr(audio_path, 0, end_time - start_time)

            # Store results
            results.append({
                'Video Link': youtube_url,
                'Timestamp': row['Timestamp'],
                'Sampling Rate (Hz)': rate,
                'SNR (dB)': snr,
            })

            print(f'Video {index+1}:')
            print(f'Sampling Rate: {rate} Hz')
            print(f'SNR: {snr} dB')
            print('-' * 40)

        except Exception as e:
            print(f"Error processing video at row {index}: {e}")
            continue

    # Save results to a CSV file
    results_df = pd.DataFrame(results)
    results_csv_path = os.path.join(output_base_path, 'results.csv')
    results_df.to_csv(results_csv_path, index=False)
    print(f'Results saved to {results_csv_path}')

if __name__ == '__main__':
    main()


[youtube] Extracting URL: https://www.youtube.com/live/YAcRXrrEFZQ?si=goQG3_tlgIwyMYHH
[youtube] YAcRXrrEFZQ: Downloading webpage
[youtube] YAcRXrrEFZQ: Downloading ios player API JSON
[youtube] YAcRXrrEFZQ: Downloading m3u8 information
Video 1:
Sampling Rate: 44100 Hz
SNR: -9.960963639683031 dB
----------------------------------------
[youtube] Extracting URL: https://youtu.be/fPlqjhYyJyg?si=MuMv-gZs9uIByfxq
[youtube] fPlqjhYyJyg: Downloading webpage
[youtube] fPlqjhYyJyg: Downloading ios player API JSON
[youtube] fPlqjhYyJyg: Downloading m3u8 information
Video 2:
Sampling Rate: 44100 Hz
SNR: -19.915173825312 dB
----------------------------------------
[youtube] Extracting URL: https://youtu.be/yJktcDKcNiE?si=qY0VnSQkiPAucKMi
[youtube] yJktcDKcNiE: Downloading webpage
[youtube] yJktcDKcNiE: Downloading ios player API JSON
[youtube] yJktcDKcNiE: Downloading m3u8 information
Video 3:
Sampling Rate: 44100 Hz
SNR: -19.758335357502897 dB
----------------------------------------
[youtube] E